In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-a-strong-stockfish-wdl-model/sample_submission.csv
/kaggle/input/train-a-strong-stockfish-wdl-model/shuffled_fens.csv
/kaggle/input/train-a-strong-stockfish-wdl-model/test.csv


In [3]:
df = pd.read_csv('/kaggle/input/train-a-strong-stockfish-wdl-model/shuffled_fens.csv')
df.columns

Index(['FEN', 'WDL'], dtype='object')

In [4]:
df.sample(10)
df.head()
df['WDL'].unique()

array([1. , 0. , 0.5])

In [ ]:
print(chess.Board('1r6/R3Pk2/6p1/2KB3p/5P2/8/8/8 b - - 5 50'))

In [5]:
!pip install python-chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 45.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=9914937b64e0012c6d3b9bf0e7c08edb3d51b3ada13cf2d9b5185a46dd50a8ea
  Stored in directory: /root/.cache/pip/wheels/fb/5d/5c/59a62d8a695285e59ec9c1f66add6f8a9ac4152499a2be0113
Successfully built chess


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import chess
import chess.engine
import chess.pgn

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from joblib import Parallel, delayed

In [7]:
def parse_fen(fen):
    board = chess.Board(fen)  # Extract the full FEN
    rows = board.board_fen().split("/")                               
    features = {
        'FEN' : board.board_fen(),
        'num_white_pawns': len(board.pieces(chess.PAWN, chess.WHITE)),
        'num_black_pawns': len(board.pieces(chess.PAWN, chess.BLACK)),
        'num_white_knights': len(board.pieces(chess.KNIGHT, chess.WHITE)),
        'num_black_knights': len(board.pieces(chess.KNIGHT, chess.BLACK)),
        'num_white_bishops': len(board.pieces(chess.BISHOP, chess.WHITE)),
        'num_black_bishops': len(board.pieces(chess.BISHOP, chess.BLACK)),
        'num_white_rooks': len(board.pieces(chess.ROOK, chess.WHITE)),
        'num_black_rooks': len(board.pieces(chess.ROOK, chess.BLACK)),
        'num_white_queens': len(board.pieces(chess.QUEEN, chess.WHITE)),
        'num_black_queens': len(board.pieces(chess.QUEEN, chess.BLACK)),
        #Castling options
        'can_white_cstl_ks' : int(board.has_kingside_castling_rights(chess.WHITE)),
        'can_white_cstl_qs' : int(board.has_queenside_castling_rights(chess.WHITE)),
        'can_black_cstl_ks' : int(board.has_kingside_castling_rights(chess.BLACK)),
        'can_black_cstl_qs' : int(board.has_queenside_castling_rights(chess.BLACK)),
        # Total pieces for each color
        'num_white_pieces': sum(len(board.pieces(piece_type, chess.WHITE)) for piece_type in chess.PIECE_TYPES),
        'num_black_pieces': sum(len(board.pieces(piece_type, chess.BLACK)) for piece_type in chess.PIECE_TYPES),
        # Board status. Each digit represents a status issue. 0 means no issues
        'status' : board.status(),
        # Number of BW turns
        'round' : board.fullmove_number,
        # Next turn. 1-white, 0-black
        'is_white_to_move': int(board.turn),
        "row1" : rows[0],
        "row2" : rows[1],
        "row3" : rows[2],
        "row4" : rows[3],
        "row5" : rows[4],
        "row6" : rows[5],
        "row7" : rows[6],
        "row8" : rows[7],
        
    }
    return features

In [29]:
# Parallelize FEN parsing
def parse_fen_parallel(fen_list, n_jobs=-1):
    return Parallel(n_jobs=n_jobs)(delayed(parse_fen)(fen) for fen in fen_list)

# Apply parallel parsing
# Subset shuffle file
# fen_list = df[0:1000000]['FEN'].tolist()
fen_list = df[0:100000]['FEN']
fen_features = parse_fen_parallel(fen_list)
features_df = pd.DataFrame(fen_features)

features_df.head()

,FEN,num_white_pawns,num_black_pawns,num_white_knights,num_black_knights,num_white_bishops,num_black_bishops,num_white_rooks,num_black_rooks,num_white_queens,...,round,is_white_to_move,row1,row2,row3,row4,row5,row6,row7,row8
0,r1b4r/1p3kb1/p2pp1p1/3q3p/3N1Pp1/2P3R1/PP1Q1BP...,6,7,1,0,1,2,2,2,1,...,23,1,r1b4r,1p3kb1,p2pp1p1,3q3p,3N1Pp1,2P3R1,PP1Q1BPP,4R1K1
1,r1bqkb1r/pp3ppp/2np1n2/8/2BNP3/8/PP3PPP/RNBQK2R,6,6,2,2,2,2,2,2,1,...,4,1,r1bqkb1r,pp3ppp,2np1n2,8,2BNP3,8,PP3PPP,RNBQK2R
2,8/R7/5k2/2pqp3/6n1/8/8/1K6,0,2,0,1,0,0,1,0,0,...,50,0,8,R7,5k2,2pqp3,6n1,8,8,1K6
3,1r6/R3Pk2/6p1/2KB3p/5P2/8/8/8,2,2,0,0,1,0,1,1,0,...,50,0,1r6,R3Pk2,6p1,2KB3p,5P2,8,8,8
4,2RB2bk/p2n3p/4Np2/8/2B3P1/1P3p1P/1P3P1K/2br4,5,4,1,1,2,2,1,1,0,...,35,1,2RB2bk,p2n3p,4Np2,8,2B3P1,1P3p1P,1P3P1K,2br4


In [30]:
#better to not select by index in case df changes.  Fix if time.
X = features_df.iloc[:, 1:19]
# y = df[0:100000]['WDL'].astype(int)

In [45]:
# map WDL values to categories
y = pd.DataFrame(df[0:100000]['WDL'])

target_encoding = {1:1, 0:0, .5:2}
y['target'] = y['WDL'].map(target_encoding)

y = pd.DataFrame(y['target'])

print(y.sample(10))

       target
38410       2
60037       1
66576       2
24681       1
1877        2
83844       2
28230       2
32825       1
18954       2
87544       1


In [21]:
# import xgboost as xgb

# # Split dataset into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train XGB model
# # xgb_model = xgb.XGBClassifier(n_estimators=200, max_depth=3, learning_rate=0.1, tree_method='gpu_hist')
# xgb_model = xgb.XGBClassifier(n_estimators=100, objective='binary:logistic', tree_method='hist', eta=0.1, max_depth=3, enable_categorical=True)
# xgb_model.fit(X_train, y_train)
    
# # Evaluate model
# y_pred = xgb_model.predict(X_test)
# print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
# print("Classification Report:\n", classification_report(y_test, y_pred))

Confusion Matrix:
 [[84 37]
 [34 45]]
Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.69      0.70       121
           1       0.55      0.57      0.56        79

    accuracy                           0.65       200
   macro avg       0.63      0.63      0.63       200
weighted avg       0.65      0.65      0.65       200



In [46]:
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats
import xgboost as xgb

# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the hyperparameter distributions
param_dist = {
    'max_depth': stats.randint(3, 10),
    'learning_rate': stats.uniform(0.01, 0.1),
    'subsample': stats.uniform(0.5, 0.5),
    'n_estimators':stats.randint(50, 200)
}

# Create the XGBoost model object
xgb_model = xgb.XGBClassifier(eta=0.1, enable_categorical=True, tree_method='hist')

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(xgb_model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy')

# Fit the RandomizedSearchCV object to the training data
random_search.fit(X_train, y_train)

# Evaluate model
y_pred = random_search.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Confusion Matrix:
 [[ 907 1635  933]
 [ 122 6604 1519]
 [ 285 3383 4612]]
Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.26      0.38      3475
           1       0.57      0.80      0.66      8245
           2       0.65      0.56      0.60      8280

    accuracy                           0.61     20000
   macro avg       0.64      0.54      0.55     20000
weighted avg       0.62      0.61      0.59     20000

